In [ ]:
import os
import glob
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import ast
import numpy as np

# Directory containing batch attack results
results_dir = 'batch_attack_results'

# 1. List all summary and per-run CSVs
all_csvs = glob.glob(os.path.join(results_dir, '*.csv'))

In [ ]:
# combine all of the per_run_csvs into a single dataframe
dfs = [pd.read_csv(f) for f in all_csvs]
df_all = pd.concat(dfs, ignore_index=True)

# Optionally remove any normal0.wav data
df_all = df_all[df_all['input_wav'] != 'processed_sound/normal0.wav']

# Convert strings to lists
for col in ['ensemble_losses', 'target_losses', 'ensemble_lev_dists', 'target_lev_dists']:
    df_all[col] = df_all[col].apply(ast.literal_eval)

print(df_all.head())
print(f"Combined DataFrame shape: {df_all.shape}")

In [ ]:
df_all

In [ ]:
# --- Target Loss ---
loss_lists = df_all["target_losses"].dropna()
max_len = max(len(lst) for lst in loss_lists)
loss_array = np.full((len(loss_lists), max_len), np.nan)
for i, lst in enumerate(loss_lists):
    loss_array[i, :len(lst)] = lst
mean_target_loss = np.nanmean(loss_array, axis=0)

# --- Ensemble Loss ---
ensemble_loss_lists = df_all["ensemble_losses"].dropna()
# Flatten to shape: (runs, ensemble_models, steps)
max_ens = max(len(run) for run in ensemble_loss_lists)
max_steps = max(len(model) for run in ensemble_loss_lists for model in run)
ens_loss_array = np.full((len(ensemble_loss_lists), max_ens, max_steps), np.nan)
for i, run in enumerate(ensemble_loss_lists):
    for j, model in enumerate(run):
        ens_loss_array[i, j, :len(model)] = model
# Mean over runs and ensemble models
mean_ensemble_loss = np.nanmean(ens_loss_array, axis=(0,1))

# --- Plot ---
plt.figure(figsize=(8,5))
plt.plot(mean_target_loss, label='Target')
plt.plot(mean_ensemble_loss, label='Ensemble')
plt.xlabel('PGD Iteration')
plt.ylabel('Mean Loss')
plt.title('Mean Loss at Each PGD Iteration (Target vs Ensemble)')
plt.legend()
plt.grid(True)
plt.show()

In [ ]:
# --- Target Lev Dist ---
target_lev_dists = df_all["target_lev_dists"].dropna()
max_len = max(len(lst) for lst in target_lev_dists)
target_lev_array = np.full((len(target_lev_dists), max_len), np.nan)
for i, lst in enumerate(target_lev_dists):
    target_lev_array[i, :len(lst)] = lst
mean_target_lev = np.nanmean(target_lev_array, axis=0)

# --- Ensemble Lev Dist ---
ensemble_lev_dists = df_all["ensemble_lev_dists"].dropna()
# Flatten to shape: (runs, ensemble_models, steps)
max_ens = max(len(run) for run in ensemble_lev_dists)
max_steps = max(len(model) for run in ensemble_lev_dists for model in run)
ens_lev_array = np.full((len(ensemble_lev_dists), max_ens, max_steps), np.nan)
for i, run in enumerate(ensemble_lev_dists):
    for j, model in enumerate(run):
        ens_lev_array[i, j, :len(model)] = model
# Mean over runs and ensemble models
mean_ensemble_lev = np.nanmean(ens_lev_array, axis=(0,1))

# --- Plot ---
plt.figure(figsize=(8,6))
plt.plot(mean_target_lev, label='Mean Target Lev Dist')
plt.plot(mean_ensemble_lev, label='Mean Ensemble Lev Dist')
plt.xlabel('PGD Iteration')
plt.ylabel('Mean Lev Dist')
plt.title('Mean Lev Dist at Each PGD Iteration (Target vs Ensemble)')
plt.legend()
plt.grid(True)
plt.show()

In [ ]:
# --- Target Lev Dist ---
target_lev_dists = df_all["target_lev_dists"].dropna()
max_len = max(len(lst) for lst in target_lev_dists)
target_lev_array = np.full((len(target_lev_dists), max_len), np.nan)
for i, lst in enumerate(target_lev_dists):
    target_lev_array[i, :len(lst)] = lst
mean_target_lev = np.nanmean(target_lev_array, axis=0)
std_target_lev = np.nanstd(target_lev_array, axis=0)

# --- Ensemble Lev Dist ---
ensemble_lev_dists = df_all["ensemble_lev_dists"].dropna()
max_ens = max(len(run) for run in ensemble_lev_dists)
max_steps = max(len(model) for run in ensemble_lev_dists for model in run)
ens_lev_array = np.full((len(ensemble_lev_dists), max_ens, max_steps), np.nan)
for i, run in enumerate(ensemble_lev_dists):
    for j, model in enumerate(run):
        ens_lev_array[i, j, :len(model)] = model
mean_ensemble_lev = np.nanmean(ens_lev_array, axis=(0,1))
std_ensemble_lev = np.nanstd(ens_lev_array, axis=(0,1))

# --- Plot ---
k = 0.5  # Change this factor as desired
x = np.arange(len(mean_target_lev))
x_ens = np.arange(len(mean_ensemble_lev))

plt.figure(figsize=(8,5))
plt.plot(mean_target_lev, label='Target')
plt.fill_between(x, mean_target_lev - k*std_target_lev, mean_target_lev + k*std_target_lev, alpha=0.2)

plt.plot(mean_ensemble_lev, label='Ensemble')
plt.fill_between(x_ens, mean_ensemble_lev - k*std_ensemble_lev, mean_ensemble_lev + k*std_ensemble_lev, alpha=0.2)

plt.xlabel('PGD Iteration')
plt.ylabel('Mean Levenshtein Distance')
plt.title('Mean Levenshtein Distance at Each PGD Iteration (Target vs Ensemble) with 0.5 std')
plt.legend()
plt.grid(True)
plt.show()

In [ ]:
# First parse out the average of the last five levenshtein distances for the target
df_all['final_target_lev_dist'] = df_all['target_lev_dists'].apply(lambda x: np.mean(x[-5:]))

# Histogram of the final target levenshtein distances
df_all['final_target_lev_dist'].hist(bins=50)
plt.show()

In [ ]:
# Extract all of the last five Levenshtein distances from each entry
last_five_lev_dists = df_all['target_lev_dists'].apply(lambda x: x[-10:])
# Flatten the list of lists into a single list
all_last_five = [item for sublist in last_five_lev_dists for item in sublist]

# Plot the histogram
plt.figure(figsize=(8, 5))
plt.hist(all_last_five, bins=10, color='skyblue', edgecolor='black')
plt.xlabel('Target Levenshtein Distance (last 10 iterations)')
plt.ylabel('Frequency')
plt.title('Histogram of All Final 10 Target Levenshtein Distances')
plt.grid(axis='y', alpha=0.75)
plt.show()

In [ ]:
# Extract all of the last ten Levenshtein distances from each entry
last_ten_lev_dists = df_all['target_lev_dists'].apply(lambda x: x[-10:])
all_last_ten = [item for sublist in last_ten_lev_dists for item in sublist]

plt.figure(figsize=(8, 5))
sns.kdeplot(all_last_ten, fill=True, color='skyblue', linewidth=2)
plt.xlabel('Target Levenshtein Distance (last 10 iterations)')
plt.ylabel('Density')
plt.title('Smoothed Distribution of All Final 10 Target Levenshtein Distances')
plt.grid(axis='y', alpha=0.75)
plt.show()

In [ ]:
# Extract all of the last ten Levenshtein distances from each entry
last_ten_lev_dists = df_all['target_lev_dists'].apply(lambda x: x[-10:])
all_last_ten = [item for sublist in last_ten_lev_dists for item in sublist]

# Calculate the mean initial Levenshtein distance (first entry of each sequence)
initial_lev_dists = df_all['target_lev_dists'].apply(lambda x: x[0])
mean_initial_lev_dist = np.mean(initial_lev_dists)

# Calculate the mean of all last ten Levenshtein distances (across all entries)
mean_last_ten_lev_dist = np.mean(all_last_ten)

plt.figure(figsize=(10, 3))
sns.kdeplot(all_last_ten, fill=True, color='skyblue', linewidth=2)
plt.axvline(mean_initial_lev_dist, color='red', linestyle='--', linewidth=2, label=f'Mean Initial Target Distance = {mean_initial_lev_dist:.2f}')
plt.axvline(mean_last_ten_lev_dist, color='green', linestyle='--', linewidth=2, label=f'Mean of Last 10 Target Distances = {mean_last_ten_lev_dist:.2f}')
plt.xlabel('Target Levenshtein Distance')
plt.ylabel('Density')
plt.title('Smoothed Distribution of The Final 10 Target Levenshtein Distances')
plt.grid(axis='y', alpha=0.75)
plt.legend()
plt.show()

In [ ]:
def plot_with_groupby(groupby_column, eval_column):
    grouped = df_all.groupby(groupby_column)
    n_groups = len(grouped)
    colors = sns.color_palette('tab10', n_colors=n_groups)

    # Collect all first and last ten distances for global x-axis range
    all_first_ten = []
    all_last_ten = []
    for _, group in grouped:
        first_ten_lev_dists = group[eval_column].apply(lambda x: x[:10])
        last_ten_lev_dists = group[eval_column].apply(lambda x: x[-10:])
        all_first_ten.extend([item for sublist in first_ten_lev_dists for item in sublist])
        all_last_ten.extend([item for sublist in last_ten_lev_dists for item in sublist])

    # Determine global min and max for x-axis
    global_min = min(min(all_first_ten), min(all_last_ten))
    global_max = max(max(all_first_ten), max(all_last_ten))

    fig, axes = plt.subplots(2, 1, figsize=(12, 10), sharey=True)

    # Panel 1: First 10
    handles_1, labels_1, means_1 = [], [], []
    for (target_val, group), color in zip(grouped, colors):
        first_ten_lev_dists = group[eval_column].apply(lambda x: x[:10])
        group_first_ten = [item for sublist in first_ten_lev_dists for item in sublist]
        if len(group_first_ten) > 1:
            mean_first = np.mean(group_first_ten)
            label = f"{target_val} (mean={mean_first:.2f})"
            line = axes[0].plot([], [], label=label)[0]  # dummy for handle
            sns.kdeplot(group_first_ten, fill=True, color=color, linewidth=2, ax=axes[0], clip=(global_min, global_max))
            handles_1.append(line)
            labels_1.append(label)
            means_1.append(mean_first)
    # Sort by mean
    sorted_1 = sorted(zip(means_1, handles_1, labels_1), key=lambda x: x[0])
    handles_1_sorted = [h for _, h, _ in sorted_1]
    labels_1_sorted = [l for _, _, l in sorted_1]
    axes[0].set_xlim(global_min, global_max)
    axes[0].set_title(f'First 10 {eval_column} by {groupby_column}')
    axes[0].set_xlabel(f'{eval_column}')
    axes[0].set_ylabel('Density')
    axes[0].grid(axis='y', alpha=0.75)
    axes[0].legend(handles=handles_1_sorted, labels=labels_1_sorted)

    # Panel 2: Last 10
    handles_2, labels_2, means_2 = [], [], []
    for (target_val, group), color in zip(grouped, colors):
        last_ten_lev_dists = group[eval_column].apply(lambda x: x[-10:])
        group_last_ten = [item for sublist in last_ten_lev_dists for item in sublist]
        if len(group_last_ten) > 1:
            mean_last = np.mean(group_last_ten)
            label = f"{target_val} (mean={mean_last:.2f})"
            line = axes[1].plot([], [], label=label)[0]  # dummy for handle
            sns.kdeplot(group_last_ten, fill=True, color=color, linewidth=2, ax=axes[1], clip=(global_min, global_max))
            handles_2.append(line)
            labels_2.append(label)
            means_2.append(mean_last)
    # Sort by mean
    sorted_2 = sorted(zip(means_2, handles_2, labels_2), key=lambda x: x[0])
    handles_2_sorted = [h for _, h, _ in sorted_2]
    labels_2_sorted = [l for _, _, l in sorted_2]
    axes[1].set_xlim(global_min, global_max)
    axes[1].set_title(f'Last 10 {eval_column} by {groupby_column}')
    axes[1].set_xlabel(f'{eval_column}')
    axes[1].grid(axis='y', alpha=0.75)
    axes[1].legend(handles=handles_2_sorted, labels=labels_2_sorted)

    plt.tight_layout()
    plt.show()

In [ ]:
for groupby_column in ('target_version', 'target_model', 'input_wav', 'target_sentence'):
    plot_with_groupby(groupby_column, 'target_lev_dists')

In [ ]:
for groupby_column in ('target_version', 'target_model', 'input_wav', 'target_sentence'):
    plot_with_groupby(groupby_column, 'target_losses')

In [ ]:
import pandas as pd
from scipy.stats import chi2_contingency

# Build the contingency table
contingency_table = pd.crosstab(df_all['input_wav'], df_all['target_model'])

# Perform the Chi-squared test
chi2, p, dof, expected = chi2_contingency(contingency_table)

print(f"Chi-squared statistic: {chi2:.2f}")
print(f"p-value: {p:.4f}")

if p < 0.05:
    print("There is a statistically significant association between input_wav and target_model.")
else:
    print("No statistically significant association was found; any apparent link is likely due to random chance.")

In [ ]:
contingency_table